In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [10]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset6') 

# Actions that we try to detect
actions = np.array(['Good', 'hello', 'Bad'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [11]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [12]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
X = np.array(sequences)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
y_test.shape

(5, 3)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [21]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])

Epoch 1/300
3/3 [==============================] - 3s 1s/step - loss: 0.5584 - categorical_accuracy: 0.7059
Epoch 2/300
3/3 [==============================] - 1s 223ms/step - loss: 0.6355 - categorical_accuracy: 0.5882
Epoch 3/300
3/3 [==============================] - 1s 237ms/step - loss: 0.7844 - categorical_accuracy: 0.5176
Epoch 4/300
3/3 [==============================] - 1s 156ms/step - loss: 0.6220 - categorical_accuracy: 0.7059
Epoch 5/300
3/3 [==============================] - 1s 188ms/step - loss: 0.5864 - categorical_accuracy: 0.6588
Epoch 6/300
3/3 [==============================] - 0s 150ms/step - loss: 0.5593 - categorical_accuracy: 0.7059
Epoch 7/300
3/3 [==============================] - 0s 166ms/step - loss: 0.5809 - categorical_accuracy: 0.6941
Epoch 8/300
3/3 [==============================] - 1s 182ms/step - loss: 0.5479 - categorical_accuracy: 0.6941
Epoch 9/300
3/3 [==============================] - 0s 176ms/step - loss: 0.5307 - categorical_accuracy: 0.6941
Epoc

3/3 [==============================] - 1s 176ms/step - loss: 0.4551 - categorical_accuracy: 0.6824
Epoch 75/300
3/3 [==============================] - 0s 151ms/step - loss: 0.4410 - categorical_accuracy: 0.7412
Epoch 76/300
3/3 [==============================] - 1s 176ms/step - loss: 0.4470 - categorical_accuracy: 0.7882
Epoch 77/300
3/3 [==============================] - 1s 204ms/step - loss: 0.4589 - categorical_accuracy: 0.6588
Epoch 78/300
3/3 [==============================] - 0s 168ms/step - loss: 0.4252 - categorical_accuracy: 0.7529
Epoch 79/300
3/3 [==============================] - 0s 146ms/step - loss: 0.4732 - categorical_accuracy: 0.7059
Epoch 80/300
3/3 [==============================] - 1s 146ms/step - loss: 0.4749 - categorical_accuracy: 0.6471
Epoch 81/300
3/3 [==============================] - 1s 216ms/step - loss: 0.4238 - categorical_accuracy: 0.8471
Epoch 82/300
3/3 [==============================] - 1s 182ms/step - loss: 0.4448 - categorical_accuracy: 0.7529
Epoch

3/3 [==============================] - 0s 127ms/step - loss: 1.0673 - categorical_accuracy: 0.3529
Epoch 147/300
3/3 [==============================] - 1s 241ms/step - loss: 1.0668 - categorical_accuracy: 0.4118
Epoch 148/300
3/3 [==============================] - 0s 167ms/step - loss: 1.0610 - categorical_accuracy: 0.4235
Epoch 149/300
3/3 [==============================] - 0s 175ms/step - loss: 1.0606 - categorical_accuracy: 0.3647
Epoch 150/300
3/3 [==============================] - 0s 126ms/step - loss: 1.0579 - categorical_accuracy: 0.4118
Epoch 151/300
3/3 [==============================] - 0s 143ms/step - loss: 1.0542 - categorical_accuracy: 0.3529
Epoch 152/300
3/3 [==============================] - 1s 207ms/step - loss: 1.0582 - categorical_accuracy: 0.4471
Epoch 153/300
3/3 [==============================] - 2s 476ms/step - loss: 1.0510 - categorical_accuracy: 0.3647
Epoch 154/300
3/3 [==============================] - 0s 171ms/step - loss: 1.0548 - categorical_accuracy: 0.31

3/3 [==============================] - 0s 158ms/step - loss: 0.9092 - categorical_accuracy: 0.6000
Epoch 219/300
3/3 [==============================] - 0s 170ms/step - loss: 0.8810 - categorical_accuracy: 0.5882
Epoch 220/300
3/3 [==============================] - 1s 186ms/step - loss: 0.8433 - categorical_accuracy: 0.6118
Epoch 221/300
3/3 [==============================] - 0s 162ms/step - loss: 0.7907 - categorical_accuracy: 0.6941
Epoch 222/300
3/3 [==============================] - 0s 163ms/step - loss: 0.7337 - categorical_accuracy: 0.7529
Epoch 223/300
3/3 [==============================] - 0s 162ms/step - loss: 0.7027 - categorical_accuracy: 0.6353
Epoch 224/300
3/3 [==============================] - 0s 129ms/step - loss: 0.7688 - categorical_accuracy: 0.6000
Epoch 225/300
3/3 [==============================] - 0s 160ms/step - loss: 0.6498 - categorical_accuracy: 0.5765
Epoch 226/300
3/3 [==============================] - 0s 138ms/step - loss: 0.7095 - categorical_accuracy: 0.62

3/3 [==============================] - 1s 166ms/step - loss: 0.4210 - categorical_accuracy: 0.8000
Epoch 291/300
3/3 [==============================] - 0s 157ms/step - loss: 0.4182 - categorical_accuracy: 0.7647
Epoch 292/300
3/3 [==============================] - 1s 177ms/step - loss: 0.3890 - categorical_accuracy: 0.8235
Epoch 293/300
3/3 [==============================] - 1s 151ms/step - loss: 0.3659 - categorical_accuracy: 0.8706
Epoch 294/300
3/3 [==============================] - 0s 160ms/step - loss: 0.3576 - categorical_accuracy: 0.9294
Epoch 295/300
3/3 [==============================] - 1s 179ms/step - loss: 0.3522 - categorical_accuracy: 0.8941
Epoch 296/300
3/3 [==============================] - 1s 181ms/step - loss: 0.3288 - categorical_accuracy: 0.9059
Epoch 297/300
3/3 [==============================] - 1s 243ms/step - loss: 0.3101 - categorical_accuracy: 0.9412
Epoch 298/300
3/3 [==============================] - 1s 215ms/step - loss: 0.2891 - categorical_accuracy: 0.95

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [27]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[4])]

'hello'

In [29]:
actions[np.argmax(y_test[4])]

'hello'

In [30]:
model.save('paper.h5')

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [32]:
yhat = model.predict(X_test)

In [33]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_multilabel_classification
from sklearn.tree import DecisionTreeClassifier

X, y = make_multilabel_classification(n_samples=1000,
                                      n_classes=3, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

tree = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

y_pred = tree.predict(X_test)

f, axes = plt.subplots(3, 5, figsize=(25, 15))
axes = axes.ravel()
for i in range(15):
    disp = ConfusionMatrixDisplay(confusion_matrix(y_test[:, i],
                                                   y_pred[:, i]),
                                  display_labels=[0, i])
    disp.plot(ax=axes[i], values_format='.4g')
    disp.ax_.set_title(f'class {i}')
    if i<10:
        disp.ax_.set_xlabel('')
    if i%5!=0:
        disp.ax_.set_ylabel('')
    disp.im_.colorbar.remove()

plt.subplots_adjust(wspace=0.10, hspace=0.1)
f.colorbar(disp.im_, ax=axes)
plt.show()

